In [ ]:
!pip install langchain langchain_openai

In [83]:
from typing import List

from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

OPENAI_API_KEY="your_api_key"

In [96]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.prefix = f"{self.name}: "
        self.message_history = ["Here is the conversation so far."]

    def speak(self) -> str:
        llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=OPENAI_API_KEY)
        message = llm(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        self.message_history.append(f"{name}: {message}")

In [97]:

class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
    ) -> None:
        self.agents = agents
        self.next_speaker = 0
        self._step = 0


    def step(self) -> tuple[str, str]:
        # 話者の選択
        speaker = self.agents[self.next_speaker]
        receiver = self.agents[1 - self.next_speaker]

        # 話者の発言
        message = speaker.speak()
        receiver.receive(speaker.name, message)

        # stepの更新
        self._step += 1

        # 話者の交代
        self.next_speaker = 1 - self.next_speaker

        return speaker.name, message

In [103]:
topic = "今週の週末は佐々木さんと田中さん二人でお出かけすることになりました。\n週末の予定を二人で話し合って決めてください"

system_message_A = SystemMessage(content=f"あなたは佐々木さんです。話し相手はあなたの旦那さんの田中さんです。{topic}")

system_message_B = SystemMessage(content=f"あなたは田中さんです。話し相手はあなたの奥さんの佐々木さんです{topic}")

In [100]:
max_iters = 6
n = 0
agents = [DialogueAgent(name="佐々木", system_message=system_message_A), DialogueAgent(name="田中", system_message=system_message_B)]

simulator = DialogueSimulator(agents=agents)
print(f"(Moderator): {topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): 今週の週末は佐々木さんと田中さん二人でお出かけすることになりました。
週末の予定を二人で話し合って決めてください


(佐々木): 田中さん、週末の予定を考えよう！何か特別な場所に行きたいですか？


(田中): 田中: そうですね、特別な場所に行ってみたいですね。どう思いますか、海に行くのはいかがでしょうか？


(佐々木): 佐々木: 海に行くのもいいですね。でも、今週末は天気があまりよくないようですね。雨が降るかもしれません。別の予定も考えておいた方がいいかもしれません。どう思いますか？


(田中): 田中: そうですね、海は楽しそうですが、天気が心配ですね。じゃあ、雨でも大丈夫な屋内のアクティビティはどうでしょうか？例えば、美術館や映画館に行くのもいいですね。あるいは、近くのカフェでゆっくりおしゃべりするのも楽しいかもしれませんね。どう思いますか？


(佐々木): 佐々木: そうですね、海もいいですが、天気が心配ですね。屋内のアクティビティも楽しそうですね。美術館や映画館もいいですが、カフェでゆっくりおしゃべりするのもいいですね。どう思いますか？雨の日でも楽しめるし、リラックスできそうですね。


(田中): 田中: そうですね、海は天気が心配ですね。カフェでゆっくりおしゃべりするのもいいですね。でも、もう少しアクティブなこともしたいですね。例えば、ボーリングやカラオケなども楽しめるかもしれません。どう思いますか？




In [104]:
topic = "今週の週末は佐々木さんと田中さん二人でお出かけすることになりました。\n週末の予定を二人で話し合って決めてください\n会話は短めでお願いします"

In [106]:
max_iters = 6
n = 0
agents = [DialogueAgent(name="佐々木", system_message=system_message_A), DialogueAgent(name="田中", system_message=system_message_B)]

simulator = DialogueSimulator(agents=agents)
print(f"(Moderator): {topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): 今週の週末は佐々木さんと田中さん二人でお出かけすることになりました。
週末の予定を二人で話し合って決めてください
会話は短めでお願いします


(佐々木): 田中さん、今週末の予定を話し合いましょうね。どこか行きたい場所ややりたいことはありますか？


(田中): 田中: そうですね、佐々木さん。今週末楽しい時間を過ごしたいですね。どこか新しいレストランに行ってみるのはどうでしょうか？それとも公園でピクニックを楽しむのもいいですね。佐々木さん、どちらがいいと思いますか？


(佐々木): 佐々木: そうですね、田中さん。新しいレストランに行ってみるのもいいですが、最近お天気が良いので、公園でピクニックもいいですね。どちらも楽しそうです。私たちはどちらかを選んで行ってみましょうか？


(田中): 田中: そうですね、佐々木さん。新しいレストランもいいですが、公園でピクニックも久しぶりで楽しそうですね。お天気も良いようですし、ピクニックにしましょうか。お弁当を作って、お気に入りの公園に行きましょう。それから、散歩したり、ボールを投げたりして楽しみましょう。どう思いますか？


(佐々木): 佐々木: そうですね、田中さん。公園でピクニック、良いアイデアだと思います。お弁当を作って、ゆっくりと楽しい時間を過ごすのは素敵ですね。お気に入りの公園に行って、散歩したりボールを投げたりして楽しんでみましょう。天気も良さそうですし、気持ちの良い週末になりそうですね。それから、公園の周りにある素敵なカフェでお茶も楽しめたらいいですね。楽しみですね、田中さん。


(田中): 田中: そうですね、佐々木さん。公園でのピクニック、素敵なアイデアですね。お弁当を作って、ゆっくりと過ごすのは心地よいですね。お気に入りの公園で散歩したり、ボールを投げたりして楽しんでみたいです。天気も良さそうですし、リフレッシュできる週末になりそうですね。カフェでお茶も楽しめるといいですね。楽しみです、佐々木さん。どんなお弁当にしましょうか？おすすめのメニューはありますか？


